# **📓 Google Colab Notebook: Perceptron & MLP Implementation**

**Install & Import Dependencies**

In [ ]:
# Basic ML libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scikit-learn for dataset and evaluation
from sklearn.datasets import load_breast_cancers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# PyTorch for Perceptron & MLP
import torch
import torch.nn as nn
import torch.optim as optim

**Load Dataset**

In [ ]:
# Load the breast cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print("Classes:", data.target_names)

Features shape: (569, 30)
Labels shape: (569,)
Classes: ['malignant' 'benign']


**Train-Test Split & Preprocessing**

In [ ]:
# Split into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize features (important for neural nets!)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

**Define a Single-Layer Perceptron**

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, input_dim):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, 1)  # single neuron

    def forward(self, x):
        return torch.sigmoid(self.fc(x))   # sigmoid activation

**Train the Perceptron**

In [ ]:
# Initialize model
input_dim = X_train.shape[1]
model_perceptron = Perceptron(input_dim)

# Loss and optimizer
criterion = nn.BCELoss()  # binary cross entropy
optimizer = optim.SGD(model_perceptron.parameters(), lr=0.01)

# Training loop
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model_perceptron(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [20/100], Loss: 0.5014
Epoch [40/100], Loss: 0.3792
Epoch [60/100], Loss: 0.3169
Epoch [80/100], Loss: 0.2785
Epoch [100/100], Loss: 0.2520


**Evaluate Perceptron**

In [ ]:
# Predictions
with torch.no_grad():
    y_pred = model_perceptron(X_test_tensor)
    y_pred_labels = (y_pred >= 0.5).int()

# Accuracy & Report
print("Perceptron Accuracy:", accuracy_score(y_test, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test, y_pred_labels))

Perceptron Accuracy: 0.9122807017543859

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.88      0.88        42
           1       0.93      0.93      0.93        72

    accuracy                           0.91       114
   macro avg       0.91      0.91      0.91       114
weighted avg       0.91      0.91      0.91       114



**Define a Multi-Layer Perceptron (MLP)**

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 32)  # first hidden layer
        self.hidden2 = nn.Linear(32, 16)         # second hidden layer
        self.output = nn.Linear(16, 1)           # output layer

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        return torch.sigmoid(self.output(x))

**Train the MLP**

In [ ]:
# Initialize model
model_mlp = MLP(input_dim)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model_mlp.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model_mlp(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [20/100], Loss: 0.5649
Epoch [40/100], Loss: 0.4057
Epoch [60/100], Loss: 0.2480
Epoch [80/100], Loss: 0.1516
Epoch [100/100], Loss: 0.1042


**Evaluate MLP**

In [ ]:
# Predictions
with torch.no_grad():
    y_pred = model_mlp(X_test_tensor)
    y_pred_labels = (y_pred >= 0.5).int()

# Accuracy & Report
print("MLP Accuracy:", accuracy_score(y_test, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test, y_pred_labels))

MLP Accuracy: 0.956140350877193

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.98      0.94        42
           1       0.99      0.94      0.96        72

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114



**Compare Perceptron vs MLP**

In [ ]:
print("Final Comparison:")
with torch.no_grad():
    acc_perceptron = accuracy_score(y_test, (model_perceptron(X_test_tensor) >= 0.5).int())
    acc_mlp = accuracy_score(y_test, (model_mlp(X_test_tensor) >= 0.5).int())

print(f"Perceptron Accuracy: {acc_perceptron:.4f}")
print(f"MLP Accuracy:        {acc_mlp:.4f}")

Final Comparison:
Perceptron Accuracy: 0.9123
MLP Accuracy:        0.9561
